# **Mount Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# **Import Library**

In [ ]:
# import os

# # Install java
# ! apt-get update -qq
# ! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
# ! java -version

In [ ]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2021-07-25 15:44:10--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-07-25 15:44:10--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1608 (1.6K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.57K  --.-KB/s    in 0s      

2021-07-25 15:44:10 (33.1 MB/s) - written to stdout [1608/1608]

setup Cola

In [ ]:
!pip install emoji

     |████████████████████████████████| 185 kB 12.0 MB/s 
  Created wheel for emoji: filename=emoji-1.4.1-py3-none-any.whl size=186393 sha256=acb5da7d8f7df8acc496c570d7ca1ecb7e4126b7ef1f088d85f6c07c2f9af96d
  Stored in directory: /root/.cache/pip/wheels/8e/68/ac/537456a5331f1405779f2b3c2a578429d2f6d7419e440330d8
Successfully built emoji


In [ ]:
import emoji
import pandas as pd
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as f
from pyspark.sql.functions import when, col
from IPython.display import display, clear_output
from pyspark.ml import PipelineModel
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
# Spark NLP
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

In [ ]:
import sparknlp
# SETTINGS
IN_PATH = "/content/drive/MyDrive/RedditData/rd*.json"
# spark = SparkSession.builder.appName("NLP").getOrCreate()
spark = sparknlp.start(gpu=False)
timestampformat = "EEE MMM dd HH:mm:ss zzzz yyyy" 
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

In [ ]:
schema = spark.read.json(IN_PATH).limit(10).schema
spark_reader = spark.readStream.schema(schema)

# **Preprocess Streaming Data**

In [ ]:
def emoji2text(text):
    return emoji.demojize(text, delimiters=(" ", " "))
udf_emoji2text = udf(emoji2text,StringType())

In [ ]:
user_regex = r"(@\w{1,15})"
hashtag_replace_regex = "#(\w{1,})"
url_regex = r"(https?:\/\/\S+|www\.\S+)"
email_regex = r"[\w.-]+@[\w.-]+.[a-zA-Z]{1,}"
# RT_regex = r"RT"
# dot_regex = r"(\.){2,}"

def cleaning_process(data):
            # Loại bỏ @Mention khỏi text
    data=(data.withColumn("text",f.regexp_replace(f.col("text"), user_regex, "")) 
            # Loại bỏ dấu # khỏi Hashtag khỏi text
            .withColumn("text",f.regexp_replace(f.col("text"), hashtag_replace_regex, "$1"))
            # Loại bỏ URL khỏi text
            .withColumn("text",f.regexp_replace(f.col("text"), url_regex, "")) 
            # Loại bỏ Email khỏi text
            .withColumn("text",f.regexp_replace(f.col("text"), email_regex, ""))
            # # Loại bỏ RT khỏi text
            # .withColumn("text",f.regexp_replace(f.col("text"), RT_regex, ""))
            # Emoji 2 text
            .withColumn('text', udf_emoji2text(f.col('text')))
            # Loại bỏ số cũng như các ký tự khỏi đoạn text
            .withColumn("text",f.regexp_replace(f.col("text"), "[^a-zA-Z]", " "))
            # Loại bỏ các khoảng trắng thừa trong câu
            .withColumn("text",f.regexp_replace(f.col("text"), " +", " "))
            # Loại vỏ các khoảng trắng đầu và cuối câu
            .withColumn("text",f.trim(f.col("text")))
            # Chuẩn hoá viết thường
            .withColumn("text",f.lower(f.col("text")))
            # Giữ lại các dòng mà đoạn text có nội dung 
            .filter(f.col("text") != ""))
    return data

In [ ]:
# Reddit
streaming_data_raw = (spark_reader.json(IN_PATH).select("created_at", "user", "text", "Pfizer", "Sinopharm", 
                                                        "Sinovac", "Moderna", "AstraZeneca", "Covaxin", "Sputnik"))

In [ ]:
streaming_data_clean = cleaning_process(streaming_data_raw)

# **Use ours pre-trained model to sentiment text**

In [ ]:
!wget https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/models/labse_xx_2.6.0_2.4_1600858075633.zip

--2021-07-25 15:48:08--  https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/models/labse_xx_2.6.0_2.4_1600858075633.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.171.157
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.171.157|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1772022983 (1.6G) [application/zip]
Saving to: ‘labse_xx_2.6.0_2.4_1600858075633.zip’

labse_xx_2.6.0_2.4_ 100%[===================>]   1.65G  33.5MB/s    in 52s     

2021-07-25 15:49:00 (32.5 MB/s) - ‘labse_xx_2.6.0_2.4_1600858075633.zip’ saved [1772022983/1772022983]



In [ ]:
!unzip /content/labse_xx_2.6.0_2.4_1600858075633.zip -d /content/pretrain_sen_emb

Archive:  /content/labse_xx_2.6.0_2.4_1600858075633.zip
  inflating: /content/pretrain_sen_emb/bert_sentence_tensorflow  
  inflating: /content/pretrain_sen_emb/fields/vocabulary/part-00010  
  inflating: /content/pretrain_sen_emb/fields/vocabulary/part-00011  
  inflating: /content/pretrain_sen_emb/fields/vocabulary/.part-00006.crc  
  inflating: /content/pretrain_sen_emb/fields/vocabulary/._SUCCESS.crc  
  inflating: /content/pretrain_sen_emb/fields/vocabulary/.part-00007.crc  
  inflating: /content/pretrain_sen_emb/fields/vocabulary/.part-00011.crc  
  inflating: /content/pretrain_sen_emb/fields/vocabulary/.part-00005.crc  
  inflating: /content/pretrain_sen_emb/fields/vocabulary/.part-00004.crc  
  inflating: /content/pretrain_sen_emb/fields/vocabulary/.part-00010.crc  
  inflating: /content/pretrain_sen_emb/fields/vocabulary/.part-00000.crc  
  inflating: /content/pretrain_sen_emb/fields/vocabulary/.part-00001.crc  
  inflating: /content/pretrain_sen_emb/fields/vocabulary/part-000

In [ ]:
# Define Spark NLP pipleline
documentAssembler = DocumentAssembler().setInputCol("text").setOutputCol("document")

embeddings = BertSentenceEmbeddings.load("/content/pretrain_sen_emb").setInputCols(["document"]).setOutputCol("sentence_embeddings")


classifierdl = ClassifierDLModel().load("/content/drive/MyDrive/bert_labse_model").setInputCols(["sentence_embeddings"]).setOutputCol("prediction")

nlpPipeline = Pipeline(
  stages = [documentAssembler,
            embeddings,
            classifierdl])

In [ ]:
# Reddit
empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

sentiment_result = pipelineModel.transform(streaming_data_clean)

result = sentiment_result.withColumn("predicted_at", f.unix_timestamp()).withColumn("created_at", col('created_at').cast("bigint"))\
                          .withColumn("timestamp", f.from_unixtime("created_at"))\
                          .withColumn("latency", col("predicted_at") - col("created_at"))

result = result.select("timestamp", "created_at", "predicted_at", "user", "text", "latency", f.explode(f.arrays_zip('document.result', 'prediction.result')).alias("cols"),
                       "Pfizer", "Sinopharm", "Sinovac", "Moderna", "AstraZeneca", "Covaxin", "Sputnik") \
                .withColumn("sentiment", f.expr("cols['1']"))\
                .withColumn('sentiment',when(col('sentiment')==0.0, 'Negative').when(col('sentiment')==1.0, 'Neutral').otherwise('Positive'))


result_count = result.groupBy("sentiment").agg(f.count("sentiment").alias("count"))

In [ ]:
result_latency = result.filter(result.latency < 50).agg(f.avg(col("latency")).alias("Avg latency"))

In [ ]:
Pfizer = result.filter(result.Pfizer == 1).groupBy("sentiment").agg(f.count("sentiment").alias("count"))
Sinopharm = result.filter(result.Sinopharm == 1).groupBy("sentiment").agg(f.count("sentiment").alias("count"))
Sinovac = result.filter(result.Sinovac == 1).groupBy("sentiment").agg(f.count("sentiment").alias("count"))
Moderna = result.filter(result.Moderna == 1).groupBy("sentiment").agg(f.count("sentiment").alias("count"))
AstraZeneca = result.filter(result.AstraZeneca == 1).groupBy("sentiment").agg(f.count("sentiment").alias("count"))
Covaxin = result.filter(result.Covaxin == 1).groupBy("sentiment").agg(f.count("sentiment").alias("count"))
Sputnik = result.filter(result.Sputnik == 1).groupBy("sentiment").agg(f.count("sentiment").alias("count"))

# **Write streaming data**

In [ ]:
stream_writer = (result.select("timestamp", "created_at", "predicted_at", "user", "text", "sentiment", "latency")\
                 .writeStream.queryName("sentiment5").trigger(processingTime="5 seconds").outputMode("append").format("memory"))
query = stream_writer.start()

In [ ]:
stream_writer_count = (result_count.writeStream.queryName("sentiment_count").trigger(processingTime="5 seconds").outputMode("complete").format("memory"))
query_count = stream_writer_count.start()

In [ ]:
stream_latency = (result_latency.writeStream.queryName("sentiment_latency").trigger(processingTime="5 seconds").outputMode("complete").format("memory"))
query_latency = stream_latency.start()

In [ ]:
stream_Pfizer = (Pfizer.writeStream.queryName("Pfizer").trigger(processingTime="5 seconds").outputMode("complete").format("memory"))
query_Pfizer = stream_Pfizer.start()

stream_Sinopharm = (Sinopharm.writeStream.queryName("Sinopharm").trigger(processingTime="5 seconds").outputMode("complete").format("memory"))
query_Sinopharm = stream_Sinopharm.start()

stream_Sinovac = (Sinovac.writeStream.queryName("Sinovac").trigger(processingTime="5 seconds").outputMode("complete").format("memory"))
query_Sinovac = stream_Sinovac.start()

stream_Moderna = (Moderna.writeStream.queryName("Moderna").trigger(processingTime="5 seconds").outputMode("complete").format("memory"))
query_Moderna = stream_Moderna.start()

stream_AstraZeneca = (AstraZeneca.writeStream.queryName("AstraZeneca").trigger(processingTime="5 seconds").outputMode("complete").format("memory"))
query_AstraZeneca = stream_AstraZeneca.start()

stream_Covaxin = (Covaxin.writeStream.queryName("Covaxin").trigger(processingTime="5 seconds").outputMode("complete").format("memory"))
query_Covaxin = stream_Covaxin.start()

stream_Sputnik = (Sputnik.writeStream.queryName("Sputnik").trigger(processingTime="5 seconds").outputMode("complete").format("memory"))
query_Sputnik = stream_Sputnik.start()

In [ ]:
if streaming_data_raw.isStreaming:
    from time import sleep
    for x in range(0,500):
        try:
            if not query.isActive:
                print("Query not active")
                break
            print("Showing live new refreshed every 5 seconds")
            print(f"Second passed: {x*5}")

            result = spark.sql(f"SELECT * from {query.name}")
            result_count = spark.sql(f"SELECT * from {query_count.name}")
            result_latency = spark.sql(f"SELECT * from {query_latency.name}")
            Pfizer = spark.sql(f"SELECT * from {query_Pfizer.name}")
            Sinopharm = spark.sql(f"SELECT * from {query_Sinopharm.name}")
            Sinovac = spark.sql(f"SELECT * from {query_Sinovac.name}")
            Moderna = spark.sql(f"SELECT * from {query_Moderna.name}")
            AstraZeneca = spark.sql(f"SELECT * from {query_AstraZeneca.name}")
            Covaxin = spark.sql(f"SELECT * from {query_Covaxin.name}")
            Sputnik = spark.sql(f"SELECT * from {query_Sputnik.name}")
            
            display(result.toPandas().sort_values(by=['created_at'], ascending=True))
            print("\n")

            result_count = result_count.toPandas()
            result_count.rename(columns={"sentiment": "All vaccine"}, inplace=True)
            result_count.sort_values(by='count', ascending=False, inplace=True)
            display(result_count)
            print("\n")
            
            display(result_latency.toPandas())
            print("\n")
            
            Pfizer = Pfizer.toPandas() 
            Pfizer['percentage'] = Pfizer['count']/Pfizer['count'].sum()
            Pfizer.rename(columns={"sentiment": "Pfizer"}, inplace=True)
            Pfizer.sort_values(by='count', ascending=False, inplace=True)
            display(Pfizer)
            print("\n")

            Sinopharm = Sinopharm.toPandas() 
            Sinopharm['percentage'] = Sinopharm['count']/Sinopharm['count'].sum()
            Sinopharm.rename(columns={"sentiment": "Sinopharm"}, inplace=True)
            Sinopharm.sort_values(by='count', ascending=False, inplace=True)
            display(Sinopharm)
            print("\n")

            Sinovac = Sinovac.toPandas() 
            Sinovac['percentage'] = Sinovac['count']/Sinovac['count'].sum()
            Sinovac.rename(columns={"sentiment": "Sinovac"}, inplace=True) 
            Sinovac.sort_values(by='count', ascending=False, inplace=True)
            display(Sinovac)
            print("\n")

            Moderna = Moderna.toPandas() 
            Moderna['percentage'] = Moderna['count']/Moderna['count'].sum()
            Moderna.rename(columns={"sentiment": "Moderna"}, inplace=True) 
            Moderna.sort_values(by='count', ascending=False, inplace=True)
            display(Moderna)
            print("\n")

            AstraZeneca = AstraZeneca.toPandas() 
            AstraZeneca['percentage'] = AstraZeneca['count']/AstraZeneca['count'].sum()
            AstraZeneca.rename(columns={"sentiment": "AstraZeneca"}, inplace=True) 
            AstraZeneca.sort_values(by='count', ascending=False, inplace=True)
            display(AstraZeneca)
            print("\n")

            Covaxin = Covaxin.toPandas() 
            Covaxin['percentage'] = Covaxin['count']/Covaxin['count'].sum()
            Covaxin.rename(columns={"sentiment": "Covaxin"}, inplace=True) 
            Covaxin.sort_values(by='count', ascending=False, inplace=True)
            display(Covaxin)
            print("\n")

            Sputnik = Sputnik.toPandas() 
            Sputnik['percentage'] = Sputnik['count']/Sputnik['count'].sum()
            Sputnik.rename(columns={"sentiment": "SputnikV"}, inplace=True) 
            Sputnik.sort_values(by='count', ascending=False, inplace=True)
            display(Sputnik)
            print("\n")

            Count = [Pfizer['count'].sum(), Sinopharm['count'].sum(), Sinovac['count'].sum(), Moderna['count'].sum(), AstraZeneca['count'].sum(), Covaxin['count'].sum(), Sputnik['count'].sum()]
            Vaccine = ['Pfizer', 'Sinopharm', 'Sinovac', 'Moderna', 'AstraZeneca', 'Covaxin', 'SputnikV']
            VC = pd.DataFrame()
            VC['Vaccine'] = Vaccine
            VC['count'] = Count
            display(VC.sort_values(by=['count'],ascending=False))
            print("\n")

            Pfizer1 = Pfizer
            Pfizer1.rename(columns={"Pfizer": "sentiment"}, inplace=True)
            Pfizer1['Vaccine'] = "Pfizer"
            Pfizer1.set_index(['Vaccine'], inplace=True)
            Sinopharm1 = Sinopharm
            Sinopharm1.rename(columns={"Sinopharm": "sentiment"}, inplace=True)
            Sinopharm1['Vaccine'] = "Sinopharm"
            Sinopharm1.set_index(['Vaccine'], inplace=True)
            Sinovac1 = Sinovac
            Sinovac1.rename(columns={"Sinovac": "sentiment"}, inplace=True)
            Sinovac1['Vaccine'] = "Sinovac"
            Sinovac1.set_index(['Vaccine'], inplace=True)
            Moderna1 = Moderna
            Moderna1.rename(columns={"Moderna": "sentiment"}, inplace=True)
            Moderna1['Vaccine'] = "Moderna"
            Moderna1.set_index(['Vaccine'], inplace=True)
            AstraZeneca1 = AstraZeneca
            AstraZeneca1.rename(columns={"AstraZeneca": "sentiment"}, inplace=True)
            AstraZeneca1['Vaccine'] = "AstraZeneca"
            AstraZeneca1.set_index(['Vaccine'], inplace=True)
            Covaxin1 = Covaxin
            Covaxin1.rename(columns={"Covaxin": "sentiment"}, inplace=True)
            Covaxin1['Vaccine'] = "Covaxin"
            Covaxin1.set_index(['Vaccine'], inplace=True)
            Sputnik1 = Sputnik
            Sputnik1.rename(columns={"SputnikV": "sentiment"}, inplace=True)
            Sputnik1['Vaccine'] = "SputnikV"
            Sputnik1.set_index(['Vaccine'], inplace=True)

            df = pd.concat([Pfizer1, Sinopharm1, Sinovac1, Moderna1, AstraZeneca1, Covaxin1, Sputnik1])
            display(df)
            print("\n")

            temp2 = df.copy().pivot_table(index = "Vaccine",columns='sentiment',values="percentage")
            temp2['total'] = df.copy().groupby(['Vaccine']).sum()['count']
            display(temp2.sort_values(by=['total'],ascending=False))

            sleep(5)
            clear_output(wait=True)
        except KeyboardInterrupt:
            print("break")
            break
    print("Live view ended...")
else:
    print("Not streaming, showing static output instead")

Showing live new refreshed every 5 seconds
Second passed: 40


,timestamp,created_at,predicted_at,user,text,sentiment,latency
483,2021-07-25 04:04:57,1627185897,1627228286,one_byte_stand,they have deemed pfizer safe the reason it s n...,Negative,42389
589,2021-07-25 04:05:08,1627185908,1627228286,FuckTERFs1717,pfizer,Neutral,42378
59,2021-07-25 04:06:54,1627186014,1627228286,veggievandam,yep factor v i ve never had a blood clot lucki...,Neutral,42272
101,2021-07-25 04:07:02,1627186022,1627228286,froo,the notion of your last paragraph is the probl...,Neutral,42264
288,2021-07-25 04:07:16,1627186036,1627228286,pablo_eskybar,that s just not possible when the federal gove...,Neutral,42250
...,...,...,...,...,...,...,...
801,2021-07-25 17:08:14,1627232894,1627233019,MoeDouglas,lol all these comments blaming the russians an...,Neutral,125
802,2021-07-25 17:08:27,1627232907,1627233019,bluedreams604,more likely sinopharm or sinovac,Neutral,112
799,2021-07-25 17:08:49,1627232929,1627233019,carolineecouture,there were clinical trials before the vaccines...,Neutral,90
800,2021-07-25 17:08:49,1627232929,1627233019,kha3288,i get migraines i also have some autoimmune li...,Neutral,90


,All vaccine,count
0,Neutral,570
1,Positive,182
2,Negative,46


,Avg latency
0,47.0


,Pfizer,count,percentage
0,Neutral,404,0.730561
1,Positive,115,0.207957
2,Negative,34,0.061483


...

,SputnikV,count,percentage
0,Neutral,32,0.820513
1,Positive,5,0.128205
2,Negative,2,0.051282


,Vaccine,count
0,Pfizer,553
3,Moderna,213
4,AstraZeneca,72
2,Sinovac,44
6,SputnikV,39
5,Covaxin,11
1,Sinopharm,10


,sentiment,count,percentage
Vaccine,,,
Pfizer,Neutral,404,0.730561
Pfizer,Positive,115,0.207957
Pfizer,Negative,34,0.061483
Sinopharm,Neutral,9,0.900000
Sinopharm,Positive,1,0.100000
Sinovac,Neutral,34,0.772727
Sinovac,Positive,8,0.181818
Sinovac,Negative,2,0.045455
Moderna,Neutral,146,0.685446


sentiment,Negative,Neutral,Positive,total
Vaccine,,,,
Pfizer,0.061483,0.730561,0.207957,553
Moderna,0.014085,0.685446,0.300469,213
AstraZeneca,0.083333,0.819444,0.097222,72
Sinovac,0.045455,0.772727,0.181818,44
SputnikV,0.051282,0.820513,0.128205,39
Covaxin,NaN,0.909091,0.090909,11
Sinopharm,NaN,0.900000,0.100000,10


break
Live view ended...
